# Introduction to py-tedopa

This package consists of different files, each serving a different purpose.

**tedopa.py** provides functions to first map the Hamiltonian of an open quantum system, linearly coupled to an environment of continuous bosonic modes, to a one dimensional chain and then perform time evolution on it.

**tmps.py** provides functions to do time evolution. They are called from within tedopa.py, but can be used standalone if desired.

In order to understand the notebooks, you should be accustomed with the notion of MPS and MPO (see [this paper](https://doi.org/10.1016/j.aop.2010.09.012) by Schollwöck). MPA (Matrix Product Array, an umbrella term for matrix product representations) is used to refer to MPS, MPO and PMPS without going into further detail. Not only is this terminology used here, also the data types `mpnum.MPArray` returned by many functions are based on these concepts. Hence, you should first get an idea of how [mpnum works](https://github.com/dseuss/mpnum/blob/master/examples/mpnum_intro.ipynb). If you want to use the TEDOPA part, you should also have knowledge of how TEDOPA and its mapping work (see [Chin et al.](http://aip.scitation.org/doi/10.1063/1.3490188)).

To help understand how to use this package, there are two separate notebooks explaining the use of the two mentioned files. The TEDOPA portion is explained [here](https://github.com/MoritzLange/py-tedopa/blob/master/examples/tedopa.ipynb), and the TMPS portion is explained [here](https://github.com/MoritzLange/py-tedopa/blob/master/examples/tmps.ipynb). When using the simulation as described in the respective notebook, there are some pitfalls to be considered. Most of them regard the TMPS algorithm, although some regard the TEDOPA algorithm. So before using either of the two, read this:

## Pitfalls
When using this program, there are a few points to keep in mind in order to get reliable results within reasonable times. In each headline it is indicated for which parts of the program these pitfalls are of importance.
#### Number of Trotter Slices [tmps, tedopa]
Due to the nature of Trotter decomposition, smaller numbers of Trotter slices will lead to higher mathematical errors. In above examples a rather small number of slices is chosen to keep the calculation time as short as possible. Smaller numbers of Trotter slices might already lead to artifacts within the plots. This can be seen when changing the number of Trotter slices to 90 in the `tedopa1` example.

Too high numbers of Trotter slices on the other hand will lead to error building up due to compression after each Trotter iteration. This needs to be avoided as well. So all in all there is a sweet spot for reliable results, which needs to be found by the user. In the models tested so far, numbers between 500 and 5000 were mostly acceptable. This however also depends on the chosen Trotter order. For higher Trotter orders, the range of slice numbers with acceptable results shifts towards smaller numbers.

#### Maximum bond dimensions in `trotter_compr` [tmps, tedopa]
In previous runs of the simulation I have noticed that, for too small allowed maximum ranks of the state during time evolution, runtime errors may occur. For way too small numbers the program might even crash throwing errors like 'SVD did not converge'. In this case, increase the allowed number of maximum ranks (and possibly decrease the relative error) in `trotter_compr`.

#### Compression for time evolution operator [tmps, tedopa]
When the time evolution operator, or parts of it, are calculated, there is also compression involved. It is specified by `compr`. As opposed to `trotter_compr`, `compr` should be chosen very small. That means that the relative error should be very small and it is best to avoid maximum ranks at all. If the time evolution operator is compressed too much, the bond dimensions of the state during time evolution will increase very fast. The less compression there is within the time evolution operators, the smaller the bond dimensions of the state during time evolution will stay and the faster the calculations will go.

#### Value of `g` [tedopa]
In many models, `g` is not only the prefactor of the recurrence coefficients used in the mapping, it is also the upper bound of the domain the spectral density is considered on. In this capacity, it is chosen so that the domain covers most of the spectral density function. Since that function often converges to 0, `g` could in theory be very high and would still satisfy said condition. On the other hand then the prefactor becomes high, which leads to higher bond dimensions during time evolution and hence larger calculation times. So `g` should be chosen as small as possible while still covering the important parts of the spectral density function.

#### Length of the chain [tedopa]
In the mathematical theory behind the mapping underlying TEDOPA, the length of the chain representing the environment is infinite. Due to practical reasons, the chain in this numerical simulation needs to be cut at some point. The energy from the system is exchanged with the chain and proceeds through its nodes. While in the infinite chain it travels away from the system and never comes back, in a finite chain it might be reflected at the end and thus lead to unwanted effects. Therefore it is a good idea to make the chain long enough.

If the chain initially has no entanglement (i.e. bond dimensions of 1), a good check would be to monitor the bond dimensions during time evolution and make sure that they stay 1 or at least small at the end of the chain. Small or non-existent entanglement suggests that there has not been much interaction between chain sites, which in turn means that this part of the chain was barely involved in the calculations.